# Melanoma detection using tranfer learning and image augmentation 

### Inside this project we will use the feature detection of a VGG16 Neural network trained into the IMAGENET dataset and image augmentaion process to increase the number of cases to be used for treining and testing 

In [2]:
# Import modules that will be used into the project 

import tensorflow as tf
import os
import zipfile
import random
from shutil import copyfile
from skimage import io
from skimage.util import random_noise
import numpy as np



/home/nireplag/anaconda3/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [3]:
# define function to create working director for data

def create_directory(source:str, object_list:list):
    """Check if the directory desired already exist into the provided path and create it otherwise"""
    for obj in object_list:
        created_path = os.path.join(source, obj)
        if os.path.exists(created_path):
            print("Directory path "+ str(created_path)+ " already exist")
        else: 
            os.mkdir(created_path)
            print("Directory " + str(created_path) + " created ")


# define function to validate if image is not corrupted and split data into test and training sets

def split_data(source:str, training:str, testing:str, split_size:float):
    """
    Function to validate if data is not corrupted and 
    split it into training and test sets. 
    """
    data_list = os.listdir(source)
    train_list = random.sample(data_list, int(len(data_list) * split_size))
    for pic in data_list:
        pic_path = os.path.join(source,pic)
        if os.path.getsize(pic_path) > 0: # file not empty/corrupted
             if pic in train_list:
                 training_path = os.path.join(training,pic)
                 copyfile(pic_path,training_path)
                 training_path = '' # clear path
             else:
                 testing_path = os.path.join(testing, pic)
                 copyfile(pic_path, testing_path)
                 testing_path = '' # clear path
        pic_path = '' # clear path
    print("Dataset cleanse and sorting completed")


# define function to make data augmentaion and save files into directory with specific tag and ID

def img_augment(source:str, methods:list = ["flipud", "fliplr", "noise"], tag:str = "aug"):
    """
    Function will use skimage and Numpy package to generate new images based of prebuilt functions
    to increase volumn of data. 
    methods: functions from skimage to be used ["flipud", "fliplr", "noise"]
    tag: added string to the end of original image
    """
    counter = 0 # init a counter to add to end of each image
    data_list = os.listdir(source)
    for pic in data_list:
        pic_path = os.path.join(source,pic)
        img = io.imread(pic_path)
        if "flipud" in methods:
            aug = np.flipud(img)
            counter = counter + 1
            img_name = str(tag) + "_" + str(counter) + "_" + str(pic) 
            io.imsave(os.path.join(source, img_name), aug)
            aug = []
            img_name = '' # clear string 
        elif "fliplr" in methods:
            aug = np.fliplr(img)
            counter = counter + 1
            img_name = str(tag) + "_" + str(counter) + "_" + str(pic) 
            io.imsave(os.path.join(source, img_name), aug)
            aug = []
            img_name = '' # clear string
        elif "noise" in methods:
            aug = random_noise(img)
            counter = counter + 1
            img_name = str(tag) + "_" + str(counter) + "_" + str(pic) 
            io.imsave(os.path.join(source, img_name), aug)
            aug = []
            img_name = '' # clear string 
        pic_path = '' # clear path
        img = []
    print(f"Data Augmentation completed fro folder {source}")

In [ ]:
# 